In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time
import datetime
from sklearn.metrics import mean_absolute_error, mean_squared_error
# Parameter windowing
input_width = 60*2
label_width = 60
shift = label_width
total_window_size = input_width + shift
OUT_STEPS = label_width
# Definisikan irisan untuk input dan label
input_slice = slice(0, input_width)
label_start = total_window_size - label_width
labels_slice = slice(label_start, None)
train_df = None


In [2]:
def  import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv'):
    
    df_begin=pd.read_csv(file_path)
    df_1=df_begin.copy()
    df_1.drop(columns=['RRU.PrbAvailDl', 'RRU.PrbAvailUl','RRU.PrbTotUl','RRU.PrbTotDl','RRC.ConnMax'], inplace=True)
    convert_time=pd.to_datetime(df_1['timestamp'], unit='s',origin='unix')
    df_1.insert(df_1.columns.get_loc('timestamp') + 1, 'datetime_column', convert_time)
    df_1.set_index('datetime_column', inplace=True)
    df_1.drop(columns=['timestamp'], inplace=True)
    
    corr_matrix = df_1.corr(numeric_only=True)[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]
    filtered_corr = corr_matrix[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']]

    # Find columns where correlation with 'RRU.PrbUsedDl' or 'RRU.PrbUsedUl' is greater than 0.5
    columns_with_high_corr = filtered_corr.index[(filtered_corr['RRU.PrbUsedDl'] > 0.5) & (filtered_corr['RRU.PrbUsedUl'] > 0.5)].tolist()

    seleted_columns = ['Viavi.Cell.Name']+ columns_with_high_corr 
    df_2= df_1[seleted_columns].copy()
    cell_name= df_2['Viavi.Cell.Name'].unique()
    df= df_2.copy()
    
    return df, cell_name

In [3]:
def standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data - mean) / std

def inverse_standardize_data(data, train_df, isoutput=True, column_output=['RRU.PrbUsedDl', 'RRU.PrbUsedUl']):
    if isoutput:
        mean = train_df[column_output].mean().values
        std = train_df[column_output].std().values
    else:
        mean = train_df.mean().values
        std = train_df.std().values

    # Reshape for broadcasting with 2D matrix
    mean = mean.reshape(1, -1)
    std = std.reshape(1, -1)

    return (data * std) + mean


In [4]:
def make_windows(data_x,data_y, total_window_size, input_slice, labels_slice):
    x = []
    y = []
    for i in range(len(data_x) - total_window_size + 1):
        window_x = data_x[i:i+total_window_size]
        x.append(window_x[input_slice])

    for i in range(len(data_y) - total_window_size + 1):
        window_y= data_y[i:i+total_window_size]
        y.append(window_y[labels_slice])

    return np.array(x), np.array(y)


In [5]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import time

def tensorflow_cnn(X_train_scaled, Y_train_scaled, X_validation_scaled, Y_validation_scaled, 
                    learning_rate, target_error, max_epochs, max_sampel_batch,
                    patience, save_best_model_path, validation_data=False, load_model=None, out_steps=OUT_STEPS):
    global model

    class MAEStopCallback(tf.keras.callbacks.Callback):
        def __init__(self, threshold):
            super(MAEStopCallback, self).__init__()
            self.threshold = threshold

        def on_epoch_end(self, epoch, logs=None):
            if logs['mae'] < self.threshold:
                print(f"\nMAE reached below {self.threshold}. Stopping training.")
                self.model.stop_training = True

                
    CONV_WIDTH = 120  # Define the width of the convolutional window
    input_width = X_train_scaled.shape[1]
    num_features = X_train_scaled.shape[2]
    num_output = Y_train_scaled.shape[2]
    out_steps = out_steps

    model = tf.keras.models.Sequential()
    if load_model is None:
        model.add(tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:, :], input_shape=(input_width, num_features)))
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=CONV_WIDTH, padding='same'))
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH/2),padding='same') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Conv1D(120, activation='relu', kernel_size=int(CONV_WIDTH),padding='valid') )
        model.add(tf.keras.layers.BatchNormalization())
        model.add(tf.keras.layers.Dense(out_steps * num_output, kernel_initializer=tf.initializers.zeros()))
        model.add(tf.keras.layers.Reshape([out_steps, num_output]))
    else:
        model = tf.keras.models.load_model(load_model)
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse', 'mae', 'mape', tf.keras.metrics.RootMeanSquaredError(name='rmse')])
    mae_stop_callback = MAEStopCallback(threshold=target_error)

    checkpoint_callback = ModelCheckpoint(
        save_best_model_path,
        monitor='val_loss',     
        mode='min',         
        save_best_only=True, 
        verbose=1            
    )

    early_stopping_callback = EarlyStopping(
        monitor='val_loss',     
        mode='min',         
        patience=patience,    
        restore_best_weights=True,
        verbose=1            
    )

    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5, patience=int(2), min_lr=0.00001, verbose=1)

    time_start = time.time()
    if validation_data:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch,  
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr], 
                  validation_data=(X_validation_scaled, Y_validation_scaled), validation_batch_size=max_sampel_batch)
    else:
        model.fit(X_train_scaled, Y_train_scaled, epochs=max_epochs, batch_size=max_sampel_batch, 
                  callbacks=[mae_stop_callback, checkpoint_callback, early_stopping_callback, reduce_lr])
    
    print("time computation seconds: ", time.time() - time_start)
    
    loss, MSE, MAE, RMSE, MAPE = model.evaluate(X_train_scaled, Y_train_scaled)
    print("loss: ", loss, "MSE: ", MSE, "MAE: ", MAE, "RMSE: ", RMSE, "MAPE: ", MAPE)
    
    return model, loss, MSE, MAE, RMSE, MAPE


In [6]:
def compute_metrics(actual_df, predictions_df):
    mae = mean_absolute_error(actual_df, predictions_df)
    mse = mean_squared_error(actual_df, predictions_df)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual_df - predictions_df) / actual_df)) * 100
    return mae, mape, mse, rmse

def process_predictions(model, x_scaled, actual_df, input_width, label_width, columns):
    predictions = model.predict(x_scaled[slice(None, None, label_width), :, :])
    predictions_reshaped = predictions.reshape(-1, len(columns))
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    predictions_df = pd.DataFrame(predictions_unscaled, columns=columns, index=actual_df[input_width:].index)
    return predictions_df
def compute_error( x_scaled, actual_df, columns, model):
    
    # Process predictions for training and validation sets
    predictions_df = process_predictions(model, x_scaled, actual_df, input_width, label_width, columns)

    display(pd.concat((predictions_df.rename(columns={columns[0]: '%s_predict'%columns[0], columns[1]: '%s_predict'%columns[1]}), actual_df[input_width:]), axis=1))
    mae, mape, mse, rmse = compute_metrics(actual_df[input_width:], predictions_df)
    
    print(f"MAE: {mae}, MAPE %: {mape}, MSE: {mse}, RMSE: {rmse}")

In [7]:

def plot_predictions(val_df, x_val_scaled, model, val_scaled, input_width, label_width, num_features):
    num_output = 2


    # Predictions for validation data
    val_predictions = model.predict(x_val_scaled[slice(None,None,label_width), :, :])
    predictions_reshaped = val_predictions.reshape(-1, num_output)
    predictions_unscaled = inverse_standardize_data(predictions_reshaped, isoutput=True, train_df=train_df)
    val_predictions_df = pd.DataFrame(predictions_unscaled, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.index)

    # Generate new predictions
    val_new = val_scaled.tail(input_width).to_numpy()
    val_new = val_new.reshape(1, -1, num_features)
    generate_predictions = model.predict(val_new)
    generate_predictions = inverse_standardize_data(generate_predictions.reshape(-1, num_output), isoutput=True, train_df=train_df)
    generate_predictions_df = pd.DataFrame(generate_predictions, columns=['RRU.PrbUsedDl', 'RRU.PrbUsedUl'], index=val_df.tail(label_width).index + pd.DateOffset(hours=1))

    # Combine predictions
    all_predictions = pd.concat([val_predictions_df, generate_predictions_df], axis=0)
    inputs_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width * 3:-label_width]
    labels_graph = val_df[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].iloc[-label_width:]
    predictions_graph = all_predictions.loc[labels_graph.index]
    new_predictions_graph = all_predictions.loc[generate_predictions_df.index]

    # Plotting
    plt.figure(figsize=(16, 8))
    for n, feature in enumerate(all_predictions.columns):
        plt.subplot(len(all_predictions.columns), 1, n + 1)
        plt.plot(inputs_graph.index, inputs_graph[feature], label='Inputs for orange', marker='.', zorder=-100, markersize=5)
        plt.plot(labels_graph.index, labels_graph[feature], label='actual output for orange and inputs for red', marker='.', zorder=-100, c='#2ca02c')
        plt.scatter(predictions_graph.index, predictions_graph[feature], marker='X', edgecolors='k', label='Prediction', c='#ff7f0e', s=int(32 * 1.5))
        plt.scatter(new_predictions_graph.index, new_predictions_graph[feature], marker='+', label='New prediction', c='#FF012D', s=int(32 * 1.5))
        plt.ylabel(feature)
        plt.legend()

    plt.subplots_adjust(hspace=0.2, top=1)
    plt.show()


In [8]:
def running_program(my_df,cell_name, index_cell, name_file, name_file_before):
    global train_df
    df_1= my_df[my_df['Viavi.Cell.Name']==cell_name[index_cell]].drop(columns=['Viavi.Cell.Name']).copy()
    df=df_1.astype(float).copy()
    n = len(df)
    train_df =df.loc[:'2023-01-07 00:00:00'].iloc[:-1]
    val_df = df['2023-01-07 00:00:00':]
    num_features = df.shape[1]
    train_scaled = standardize_data(train_df, isoutput=False, train_df=train_df)
    val_scaled= standardize_data(val_df, isoutput=False, train_df=train_df)
    # Membuat windowed dataset untuk set pelatihan, validasi, dan pengujian

    x_train_scaled, y_train_scaled = make_windows(train_scaled.to_numpy(), train_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)
    x_val_scaled, y_val_scaled = make_windows(val_scaled.to_numpy(), val_scaled[['RRU.PrbUsedDl', 'RRU.PrbUsedUl']].to_numpy(),total_window_size, input_slice, labels_slice)

    if index_cell==0:
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=20, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=None, out_steps=OUT_STEPS)
        model.summary()
        model.save(name_file)
    else:
        print('file before: ', name_file_before)
        model, loss, MSE, MAE, RMSE,  MAPE  = tensorflow_cnn(x_train_scaled, y_train_scaled, x_val_scaled, y_val_scaled,
                                                        learning_rate=0.001, target_error=0.001,  max_epochs=20, max_sampel_batch=60, 
                                                        patience=6,  save_best_model_path = name_file, 
                                                        validation_data=True, load_model=name_file_before, out_steps=OUT_STEPS)
    
    model = tf.keras.models.load_model('%s'%name_file)
    columns = ['RRU.PrbUsedDl', 'RRU.PrbUsedUl']
    output_actual_train = train_df[columns]
    output_actual_val = val_df[columns]
    print("Training Metrics:")
    compute_error(x_train_scaled,  output_actual_train,columns, model)
    print("\nValidation Metrics:")
    compute_error(x_val_scaled,  output_actual_val,columns, model)
    #print("Training Plot:")
    #plot_predictions( output_actual_train[input_width:], x_train_scaled, model, train_scaled, input_width, label_width, num_features)
    #print("\nValidation (test) Plot:")
    #plot_predictions( output_actual_val[input_width:], x_val_scaled, model, val_scaled, input_width, label_width, num_features)
    


In [9]:
df_start, cell_name= import_data(file_path='D:\\KULIAH\\teep\AI\\dataset\\52 cells\\CellReports.csv')
now = datetime.datetime.now()
timestamp = now.strftime("%Y%m%d_%H%M")
skip_indices = [26, 29, 31, 37, 40]
for index in range(0,52):
    print(index)
    if index in skip_indices:
        print(index)
        continue
    cells_name= df_start['Viavi.Cell.Name'].unique()
    df_filtered = df_start[df_start['Viavi.Cell.Name'] == cells_name[index]]
    df_no_duplicates = df_filtered.loc[~df_filtered.index.duplicated()]
    name_file='1b_%s_%s.hdf5'%(timestamp, index+1)
    name_file_before='1b_%s_%s.hdf5'%(timestamp, index)
    if index in np.array(skip_indices)+1:
        name_file_before='1b_%s_%s.hdf5'%(timestamp, index-1)
    print("name_file: ", name_file)
    print("name_file_before: ", name_file_before)
    print("Cell Name: ", cell_name[index])
    running_program(my_df=df_no_duplicates, cell_name=cell_name, index_cell=index, name_file=name_file, name_file_before=name_file_before)

0
name_file:  1b_20240724_1811_1.hdf5
name_file_before:  1b_20240724_1811_0.hdf5
Cell Name:  S1/B2/C1
Epoch 1/20
150/150 [==============================] - ETA: 0s - loss: 0.7148 - mse: 0.7148 - mae: 0.5848 - mape: 213.1528 - rmse: 0.8455
Epoch 1: val_loss improved from inf to 0.88951, saving model to 1b_20240724_1811_1.hdf5
150/150 [==============================] - 9s 23ms/step - loss: 0.7148 - mse: 0.7148 - mae: 0.5848 - mape: 213.1528 - rmse: 0.8455 - val_loss: 0.8895 - val_mse: 0.8895 - val_mae: 0.5859 - val_mape: 147.9252 - val_rmse: 0.9431 - lr: 0.0010
Epoch 2/20
150/150 [==============================] - ETA: 0s - loss: 0.3573 - mse: 0.3573 - mae: 0.4332 - mape: 188.4012 - rmse: 0.5978
Epoch 2: val_loss did not improve from 0.88951
150/150 [==============================] - 4s 23ms/step - loss: 0.3573 - mse: 0.3573 - mae: 0.4332 - mape: 188.4012 - rmse: 0.5978 - val_loss: 1.4110 - val_mse: 1.4110 - val_mae: 0.9258 - val_mape: 390.2015 - val_rmse: 1.1879 - lr: 0.0010
Epoch 3/20


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 18:00:00,98.193992,98.193992,98.333333,98.333333
2022-12-31 18:01:00,100.417561,100.417561,100.000000,100.000000
2022-12-31 18:02:00,74.892935,74.892935,75.083333,75.083333
2022-12-31 18:03:00,100.579574,100.579574,100.000000,100.000000
2022-12-31 18:04:00,100.013353,100.013353,100.000000,100.000000
...,...,...,...,...
2023-01-06 23:55:00,98.279932,98.279932,98.066667,98.066667
2023-01-06 23:56:00,96.717829,96.717829,97.233333,97.233333
2023-01-06 23:57:00,94.039938,94.039938,93.816667,93.816667


MAE: 0.30969595774129755, MAPE %: 0.39078503135423304, MSE: 0.1617582529795886, RMSE: 0.4021918111791793

Validation Metrics:
1/1 [==============================] - 0s 41ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 02:00:00,96.371685,96.371685,93.950000,93.950000
2023-01-07 02:01:00,97.221454,97.221454,100.000000,100.000000
2023-01-07 02:02:00,94.723737,94.723737,96.650000,96.650000
2023-01-07 02:03:00,86.808551,86.808551,72.933333,72.933333
2023-01-07 02:04:00,100.320831,100.320831,100.000000,100.000000
...,...,...,...,...
2023-01-07 15:55:00,87.426844,87.426844,63.333333,63.333333
2023-01-07 15:56:00,89.680711,89.680711,100.000000,100.000000
2023-01-07 15:57:00,90.295093,90.295093,100.000000,100.000000


MAE: 7.3355013583978685, MAPE %: 10.623723376936216, MSE: 125.88754980531432, RMSE: 11.21996211247232
1
name_file:  1b_20240724_1811_2.hdf5
name_file_before:  1b_20240724_1811_1.hdf5
Cell Name:  S7/B2/C1
file before:  1b_20240724_1811_1.hdf5
Epoch 1/20
148/150 [============================>.] - ETA: 0s - loss: 0.2676 - mse: 0.2676 - mae: 0.3842 - mape: 129.1742 - rmse: 0.5173
Epoch 1: val_loss improved from inf to 0.59478, saving model to 1b_20240724_1811_2.hdf5
150/150 [==============================] - 4s 22ms/step - loss: 0.2663 - mse: 0.2663 - mae: 0.3830 - mape: 128.8107 - rmse: 0.5160 - val_loss: 0.5948 - val_mse: 0.5948 - val_mae: 0.6466 - val_mape: 199.2156 - val_rmse: 0.7712 - lr: 0.0010
Epoch 2/20
148/150 [============================>.] - ETA: 0s - loss: 0.0430 - mse: 0.0430 - mae: 0.1646 - mape: 60.4037 - rmse: 0.2073
Epoch 2: val_loss improved from 0.59478 to 0.50725, saving model to 1b_20240724_1811_2.hdf5
150/150 [==============================] - 3s 21ms/step - loss: 0.

...

,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2022-12-31 18:00:00,8.716857,8.716847,7.483333,7.483333
2022-12-31 18:01:00,23.003159,23.003162,22.483333,22.483333
2022-12-31 18:02:00,0.042543,0.042541,0.000000,0.000000
2022-12-31 18:03:00,-0.070925,-0.070936,0.000000,0.000000
2022-12-31 18:04:00,-0.268591,-0.268592,0.000000,0.000000
...,...,...,...,...
2023-01-06 23:55:00,13.690678,13.690683,13.116667,13.116667
2023-01-06 23:56:00,4.729001,4.728996,5.533333,5.533333
2023-01-06 23:57:00,0.215070,0.215082,0.000000,0.000000


MAE: 0.3893638689175197, MAPE %: inf, MSE: 0.23964706907004446, RMSE: 0.48953760741136576

Validation Metrics:
1/1 [==============================] - 0s 18ms/step


,RRU.PrbUsedDl_predict,RRU.PrbUsedUl_predict,RRU.PrbUsedDl,RRU.PrbUsedUl
datetime_column,,,,
2023-01-07 02:00:00,-0.958367,-0.958353,0.000000,0.000000
2023-01-07 02:01:00,5.644511,5.644514,7.500000,7.500000
2023-01-07 02:02:00,14.297613,14.297617,3.750000,3.750000
2023-01-07 02:03:00,21.044581,21.044581,2.500000,2.500000
2023-01-07 02:04:00,30.341599,30.341611,16.233333,16.233333
...,...,...,...,...
2023-01-07 15:55:00,3.327426,3.327426,27.500000,27.500000
2023-01-07 15:56:00,9.971207,9.971207,4.000000,4.000000
2023-01-07 15:57:00,21.856370,21.856381,32.166667,32.166667


MAE: 9.023649290612648, MAPE %: inf, MSE: 155.23668029613717, RMSE: 12.459401281608084
29
name_file:  1b_20240724_1811_30.hdf5
name_file_before:  1b_20240724_1811_29.hdf5
Cell Name:  S9/B13/C2
file before:  1b_20240724_1811_29.hdf5
Epoch 1/20
150/150 [==============================] - ETA: 0s - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan
Epoch 1: val_loss did not improve from inf
150/150 [==============================] - 3s 18ms/step - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan - val_loss: nan - val_mse: nan - val_mae: nan - val_mape: nan - val_rmse: nan - lr: 0.0010
Epoch 2/20
148/150 [============================>.] - ETA: 0s - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan
Epoch 2: val_loss did not improve from inf

Epoch 2: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
150/150 [==============================] - 2s 16ms/step - loss: nan - mse: nan - mae: nan - mape: nan - rmse: nan - val_loss: nan - val_mse: nan - val_mae: nan 

OSError: No file or directory found at 1b_20240724_1811_30.hdf5